<a href="https://colab.research.google.com/github/william2215/Python_Yelp_Request_Data_Cleaning/blob/main/Python_Yelp_Request_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [608]:
import pandas as pd
import requests
from types import SimpleNamespace
import json
f = open("token.txt", "r")

class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r

def cleanUpTransactions(x):
  x['transactions'] = ','.join(map(str, x['transactions']))

def cleanUpCoordinates(x):
  x['latitude'] =  x['coordinates']['latitude']
  x['longitude'] =  x['coordinates']['longitude']
  del x['coordinates']

def cleanUpLocation(x):
  dictionaryToAdd = {k:v for k,v in x['location'].items()}  
  del x['location']
  x.update(dictionaryToAdd)
  x['display_address'] = ';'.join(map(str, x['display_address']))

def cleanUpCategories(datosToExport):
  listaCategories = []
  for x in datosToExport['categories']:
    lista = []
    for z in x:
      lista.append(z['title']+' ('+z['alias']+')')
    listaCategories.append(lista)
  finalList = []
  for x in listaCategories:
    finalList.append(' '.join(map(str, x)))
  datosToExport['categories'] = finalList

def procesar():
  for x in data['businesses']:
    cleanUpTransactions(x)
    cleanUpCoordinates(x)
    cleanUpLocation(x)
    dictionary = {k:v for k,v in x.items()}  
    dataFrame.append(dictionary)

def procesarUnico(data):
    cleanUpTransactions(data)
    cleanUpCoordinates(data)
    cleanUpLocation(data)
    dictionary = {k:v for k,v in data.items()}  
    dataFrame.append(dictionary)

def timeConvert(miliTime):
    hours = miliTime[0]+miliTime[1]
    minutes = miliTime[2]+miliTime[3]
    hours, minutes = int(hours), int(minutes)
    setting = "AM"
    if hours > 12:
        setting = "PM"
        hours -= 12
    return ("%02d:%02d" + setting) % (hours, minutes)


In [609]:
response = requests.get('https://api.yelp.com/v3/businesses/search?location=Asheville&categories=Restaurants',
                        auth=BearerAuth(f.read()))
data = response.json()
dataFrame = []
procesar()
datosToExport = pd.DataFrame(dataFrame)
cleanUpCategories(datosToExport)
datosToExport

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,phone,display_phone,distance,latitude,longitude,address1,address2,address3,city,zip_code,country,state,display_address
0,Yq7j28Ws9TbM551SjkML8A,tupelo-honey-asheville,Tupelo Honey,https://s3-media3.fl.yelpcdn.com/bphoto/tAyyBb...,False,https://www.yelp.com/biz/tupelo-honey-ashevill...,3058,Southern (southern) Breakfast & Brunch (breakf...,4.5,"delivery,pickup",$$,+18282554863,(828) 255-4863,1316.960464,35.594820,-82.554889,12 College St,,,Asheville,28801,US,NC,"12 College St;Asheville, NC 28801"
1,xlAw9sqb9RV8NJZ3MVPG5g,sunny-point-café-asheville,Sunny Point Café,https://s3-media2.fl.yelpcdn.com/bphoto/poJJZJ...,False,https://www.yelp.com/biz/sunny-point-caf%C3%A9...,1950,Southern (southern) Breakfast & Brunch (breakf...,4.5,delivery,$$,+18282520055,(828) 252-0055,3003.793786,35.578590,-82.588850,626 Haywood Rd,,,Asheville,28806,US,NC,"626 Haywood Rd;Asheville, NC 28806"
2,a_6wDiDT_H58pVfwC3dCLw,cúrate-asheville-2,Cúrate,https://s3-media2.fl.yelpcdn.com/bphoto/rXttVK...,False,https://www.yelp.com/biz/c%C3%BArate-asheville...,1861,Tapas Bars (tapas) Spanish (spanish) Wine Bars...,4.5,delivery,$$$,+18282392946,(828) 239-2946,1306.939883,35.594200,-82.551620,13 Biltmore Ave,,,Asheville,28801,US,NC,"13 Biltmore Ave;Asheville, NC 28801"
3,qwWIMTXn7LR2eiCjeHRHJA,biscuit-head-asheville-2,Biscuit Head,https://s3-media2.fl.yelpcdn.com/bphoto/MOGBhS...,False,https://www.yelp.com/biz/biscuit-head-ashevill...,1643,Cafes (cafes) American (New) (newamerican) Bre...,4.5,delivery,$$,+18285053449,(828) 505-3449,454.306831,35.581281,-82.551528,417 Biltmore Ave,Ste 4F,,Asheville,28801,US,NC,"417 Biltmore Ave;Ste 4F;Asheville, NC 28801"
4,8AghPgn8HRDjyCgYeHmHow,white-duck-taco-shop-asheville,White Duck Taco Shop,https://s3-media4.fl.yelpcdn.com/bphoto/YrEe4M...,False,https://www.yelp.com/biz/white-duck-taco-shop-...,1724,Tacos (tacos),4.5,delivery,$,+18282541398,(828) 254-1398,1910.104111,35.594026,-82.572290,388 Riverside Dr,,None,Asheville,28801,US,NC,"388 Riverside Dr;Asheville, NC 28801"
5,iC-SSZJhO0EHREOvQqnuIA,hole-hot-doughnuts-and-fresh-coffee-asheville,HOLE - Hot Doughnuts and Fresh Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/9MaYLj...,False,https://www.yelp.com/biz/hole-hot-doughnuts-an...,617,Donuts (donuts) Coffee & Tea (coffee),5.0,delivery,$$,+18287745667,(828) 774-5667,1576.273896,35.580770,-82.573130,168 Haywood Rd,,,Asheville,28806,US,NC,"168 Haywood Rd;Asheville, NC 28806"
6,oIiAA7wU1_YUq_wo8kinBQ,biscuit-head-asheville,Biscuit Head,https://s3-media3.fl.yelpcdn.com/bphoto/-cQngR...,False,https://www.yelp.com/biz/biscuit-head-ashevill...,1002,Bakeries (bakeries) Breakfast & Brunch (breakf...,4.5,delivery,$,+18283335145,(828) 333-5145,3372.460620,35.577880,-82.592770,733 Haywood Rd,,,Asheville,28806,US,NC,"733 Haywood Rd;Asheville, NC 28806"
7,3r4dxgOOTT9WoDFIcQqpuw,chai-pani-asheville,Chai Pani,https://s3-media1.fl.yelpcdn.com/bphoto/R0j5i3...,False,https://www.yelp.com/biz/chai-pani-asheville?a...,991,Indian (indpak),4.5,delivery,$$,+18282544003,(828) 254-4003,1338.150331,35.595036,-82.555363,22 Battery Park Ave,,,Asheville,28801,US,NC,"22 Battery Park Ave;Asheville, NC 28801"
8,mhAPcr9dMwmai36EB06cOw,homegrown-asheville,HomeGrown,https://s3-media3.fl.yelpcdn.com/bphoto/5GRUHC...,False,https://www.yelp.com/biz/homegrown-asheville?a...,1095,American (New) (newamerican) Salad (salad) Win...,4.5,delivery,$,+18282324340,(828) 232-4340,3192.971310,35.611713,-82.554833,371 Merrimon Ave,,,Asheville,28801,US,NC,"371 Merrimon Ave;Asheville, NC 28801"
9,d8Vg0DxRY-s2a8xnZ6ratw,chestnut-asheville,Chestnut,https://s3-media1.fl.yelpcdn.com/bphoto/UdiCkh...,False,https://www.yelp.com/biz/chestnut-asheville?ad...,1001,American (New) (newamerican) Southern (southern),4.5,"delivery,pickup",$$,+18285752667,(828) 575-2667,1220.348210,35.593166,-82.550943,48 Biltmore Ave,,,Asheville,28801,US,NC,"48 Biltmore Ave;Asheville, NC 28801"


In [623]:
f = open("token.txt", "r")
field = input("")
response = requests.get('https://api.yelp.com/v3/businesses/'+field,
                        auth=BearerAuth(f.read()))
data = response.json()
data
dataFrame = []
procesarUnico(data)
datosToExport = pd.DataFrame(dataFrame)
cleanUpCategories(datosToExport)
datosToExport['hours_type'] =  datosToExport['hours'][0][0]['hours_type']
datosToExport['is_open_now'] = datosToExport['hours'][0][0]['is_open_now']
def week(i):
        switcher={0:'Sunday',
                1:'Monday',
                2:'Tuesday',
                3:'Wednesday',
                4:'Thursday',
                5:'Friday',
                6:'Saturday'}
        return switcher.get(i,"Invalid day of week")
for x in datosToExport['hours'][0][0]['open']:
  datosToExport[week(x['day'])+ " is_overnight"] = x["is_overnight"]
  datosToExport[week(x['day'])+ " start"] = timeConvert(x["start"])
  datosToExport[week(x['day'])+ " end"] = timeConvert(x["end"])
del datosToExport['hours']
datosToExport


mhAPcr9dMwmai36EB06cOw


,id,alias,name,image_url,is_claimed,is_closed,url,phone,display_phone,review_count,categories,rating,photos,price,transactions,latitude,longitude,address1,address2,address3,city,zip_code,country,state,display_address,cross_streets,hours_type,is_open_now,Monday is_overnight,Monday start,Monday end,Tuesday is_overnight,Tuesday start,Tuesday end,Wednesday is_overnight,Wednesday start,Wednesday end,Thursday is_overnight,Thursday start,Thursday end,Friday is_overnight,Friday start,Friday end,Saturday is_overnight,Saturday start,Saturday end
0,mhAPcr9dMwmai36EB06cOw,homegrown-asheville,HomeGrown,https://s3-media3.fl.yelpcdn.com/bphoto/5GRUHC...,True,False,https://www.yelp.com/biz/homegrown-asheville?a...,+18282324340,(828) 232-4340,1095,American (New) (newamerican) Salad (salad) Win...,4.5,[https://s3-media3.fl.yelpcdn.com/bphoto/5GRUH...,$,delivery,35.611713,-82.554833,371 Merrimon Ave,,,Asheville,28801,US,NC,"371 Merrimon Ave;Asheville, NC 28801",,REGULAR,False,False,11:00AM,09:00PM,False,11:00AM,09:00PM,False,11:00AM,09:00PM,False,11:00AM,09:00PM,False,11:00AM,09:00PM,False,11:00AM,09:00PM
